In [1]:
import numpy as np
import pandas as pd

In [2]:
cols = [
    "objectid",
    "year",
    "district",
    "date_",
    "primary_st",
    "age",
    "sex",
    "crash_type",
    "hit_____ru",
    "veh1",
    "veh2",
    "arrest_yes",
    "investigat",
    "time_of_day__24hrs_",
    "lat",
    "lng",
]

In [3]:
df = pd.read_csv(
    "https://phl.carto.com/api/v2/sql?filename=fatal_crashes&format=csv&skipfields=cartodb_id,the_geom,the_geom_webmercator&q=SELECT%20*,%20ST_Y(the_geom)%20AS%20lat,%20ST_X(the_geom)%20AS%20lng%20FROM%20fatal_crashes",
    usecols=cols,
    index_col=False,
)
column_mapping = {
    "objectid": "crash_id",
    "district": "police_district",
    "date_": "crash_date",
    "primary_st": "primary_street",
    "sex": "victim_gender",
    "crash_type": "collision_type",
    "hit_____ru": "hit_and_run",
    "veh1": "primary_vehicle",
    "veh2": "secondary_vehicle",
    "arrest_yes": "arrest_made",
    "investigat": "investigation_status",
    "time_of_day__24hrs_": "time",
    "lat": "latitude",
    "lng": "longitude",
}
df = df.rename(columns=column_mapping)

In [4]:
df.sample(3)

,crash_id,year,police_district,crash_date,primary_street,age,victim_gender,collision_type,hit_and_run,primary_vehicle,secondary_vehicle,arrest_made,investigation_status,time,latitude,longitude
483,49987,2022,2,2022-10-14 04:00:00+00,Verree Rd.,63.0,M,The pedestrian was attempting to cross Verree ...,Yes,Auto,Pedestrian,Pending,Pending identification of operator.,NaN,40.070730,-75.078562
352,49857,2021,39,2021-10-29 04:00:00+00,Hunting Park Ave.,56.0,F,Pedestrian attempting to cross Hunting Park Av...,Yes,Auto,Pedestrian,Pending,Pending identification,NaN,40.009911,-75.166874
440,49945,2022,16,2022-06-22 04:00:00+00,44th St.,16.0,M,"Unit #1, in a stolen vehicle S/B on 44th St. a...",No,Auto,Pedestrian,Yes,"HBV, Inv. Mans., Accident involving death, REA...",NaN,39.966775,-75.211110


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   crash_id              797 non-null    int64  
 1   year                  797 non-null    int64  
 2   police_district       797 non-null    int64  
 3   crash_date            797 non-null    object 
 4   primary_street        797 non-null    object 
 5   age                   792 non-null    float64
 6   victim_gender         796 non-null    object 
 7   collision_type        797 non-null    object 
 8   hit_and_run           797 non-null    object 
 9   primary_vehicle       797 non-null    object 
 10  secondary_vehicle     794 non-null    object 
 11  arrest_made           781 non-null    object 
 12  investigation_status  793 non-null    object 
 13  time                  292 non-null    object 
 14  latitude              783 non-null    float64
 15  longitude             7

In [6]:
df.isnull().sum()

crash_id                  0
year                      0
police_district           0
crash_date                0
primary_street            0
age                       5
victim_gender             1
collision_type            0
hit_and_run               0
primary_vehicle           0
secondary_vehicle         3
arrest_made              16
investigation_status      4
time                    505
latitude                 14
longitude                14
dtype: int64

In [7]:
df.describe()

,crash_id,year,police_district,age,latitude,longitude
count,797.000000,797.000000,797.000000,792.000000,783.000000,783.000000
mean,49904.000000,2021.712673,18.398996,43.381313,40.011338,-75.156402
std,230.218375,1.749085,10.885373,18.796025,0.145391,0.337886
min,49506.000000,2019.000000,1.000000,0.000000,39.884438,-80.502372
25%,49705.000000,2020.000000,9.000000,29.000000,39.974730,-75.179779
50%,49904.000000,2022.000000,17.000000,40.000000,40.005485,-75.144203
75%,50103.000000,2023.000000,25.000000,58.000000,40.032948,-75.097440
max,50302.000000,2025.000000,77.000000,99.000000,42.224336,-74.961302


In [8]:
df["crash_date"] = df["crash_date"].str.split().str[0]
df["crash_date"] = pd.to_datetime(df["crash_date"])

In [9]:
df.insert(
    loc=df.columns.get_loc("crash_date") + 1,
    column="day_name",
    value=df["crash_date"].dt.day_name(),
)

In [10]:
df["age_group"] = pd.cut(
    df["age"],
    bins=[0, 18, 25, 35, 45, 55, 65, 75, 100],
    labels=["0-18", "19-25", "26-35", "36-45", "46-55", "56-65", "66-75", "75+"],
)

In [11]:
df["hit_and_run"].value_counts()

hit_and_run
No     601
Yes    194
Unk      1
??       1
Name: count, dtype: int64

In [12]:
df["hit_and_run"] = np.where(
    df["hit_and_run"] == "Yes", 1, np.where(df["hit_and_run"] == "No", 0, np.nan)
)

In [13]:
df["arrest_made"].value_counts()

arrest_made
No               550
Pending          147
Yes               73
No                 4
TBD                2
Pending ID         1
Unk.               1
Arrested           1
None expected      1
N                  1
Name: count, dtype: int64

In [14]:
day_order = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]
df["day_name"] = pd.Categorical(df["day_name"], categories=day_order, ordered=True)

In [15]:
df["victim_gender"].value_counts()

victim_gender
M     577
F     215
Mn      1
MN      1
f       1
M.      1
Name: count, dtype: int64

In [16]:
gender_mapping = {
    "M": "M",
    "F": "F",
    "MN": "M",
    "M.": "M",
    "Mn": "M",
    "f": "F",
}

df["victim_gender"] = df["victim_gender"].map(gender_mapping)
df["victim_gender"].value_counts()

victim_gender
M    580
F    216
Name: count, dtype: int64

In [17]:
df["primary_vehicle"].value_counts()

primary_vehicle
Auto                  624
M/C                    78
Motorcycle             29
Auto                   10
Dirtbike                9
Train                   9
Bus                     4
ATV                     4
Quad                    2
Bicyclist               2
Bike                    2
SEPTA Bus               2
Motor Scooter           2
Dirt Bike               2
Auto (Mack TK)          1
School Bus              1
Tree                    1
PFD Ladder TK           1
SEPTA Trolley           1
SEPTA Subway Train      1
Ambulance               1
Pedestrian              1
Tow truck               1
Van                     1
T/T                     1
Moped                   1
Septa bus               1
Truck                   1
Trolley Car             1
unk                     1
Auto (Police)           1
Can Am Slingshot        1
Name: count, dtype: int64

In [18]:
vehicle_mapping = {
    "Dirt Bike": "Dirtbike",
    "SEPTA Bus": "Bus",
    "Septa bus": "Bus",
    "Motor Scooter": "Moped",
    "T/T": "Truck",
    "Auto (Police)": "Auto",
    "Auto (Mack TK)": "Auto",
    "SEPTA Trolley": "Trolley",
    "SEPTA Subway Train": "Train",
    "PFD Ladder TK": "Truck",
    "Can Am Slingshot": "ATV",
}
df["primary_vehicle"] = df["primary_vehicle"].replace(vehicle_mapping)
vehicle_categories = {
    "M/C": "Motorcycle",
    "Auto": "Auto",
    "Motorcycle": "Motorcycle",
    "Dirtbike": "Motorcycle",
    "Bike": "Bicycle",
    "Bicyclist": "Bicycle",
    "Bus": "Commercial",
    "School Bus": "Commercial",
    "Truck": "Commercial",
    "Tow truck": "Commercial",
    "Ambulance": "Commercial",
    "Train": "Train",
    "Trolley": "Trolley",
    "ATV": "Off-road",
    "Quad": "Off-road",
    "Moped": "Motorcycle",
    "Van": "Auto",
    "Pedestrian": "Pedestrian",
    "Tree": "Object",
}
df["primary_vehicle"] = df["primary_vehicle"].map(vehicle_categories)


unmapped = df[df["primary_vehicle"].isna()]["primary_vehicle"].unique()


df["primary_vehicle"] = df["primary_vehicle"].fillna("Others")
print(df["primary_vehicle"].value_counts())

primary_vehicle
Auto          627
Motorcycle    121
Commercial     13
Others         12
Train          10
Off-road        7
Bicycle         4
Pedestrian      1
Trolley         1
Object          1
Name: count, dtype: int64


In [19]:
df["secondary_vehicle"].value_counts()

secondary_vehicle
Pedestrian               316
Auto                     234
Fixed Object             137
Bicycle                   17
Bicyclist                 10
M/C                        9
Parked veh.                8
Tree                       6
Parked                     4
Scooter                    4
Bus                        4
Motorcycle                 3
E-Scooter                  3
Ground                     3
Parked Trailer             3
Fixed object               3
Dirt-bike                  3
Front-end loader           2
M/C and PED                2
Auto (Parked)              2
Trolley car                2
Parked auto                2
Tractor-Trailer            2
Mini-bike                  2
Truck                      1
Parked autos               1
Pedestrian on scooter      1
ATV                        1
T/T                        1
E-Bicycle                  1
School Bus                 1
Dirtbike                   1
Ped on skateboard          1
Moped                    

In [20]:
standardization_map = {
    "Pedestrian on scooter": "Pedestrian",
    "Ped on skateboard": "Pedestrian",
    "Pedestrians": "Pedestrian",
    "Parked veh.": "Parked Vehicle",
    "Parked auto": "Parked Vehicle",
    "Parked autos": "Parked Vehicle",
    "Auto (Parked)": "Parked Vehicle",
    "Parked": "Parked Vehicle",
    "Parked TT": "Parked Vehicle",
    "T/T": "Truck",
    "Tractor-Trailer": "Truck",
    "Dirt-bike": "Dirtbike",
    "Mini-bike": "Dirtbike",
    "E-Scooter": "Scooter",
    "E-Bicycle": "Bicycle",
    "Bicyclist": "Bicycle",
    "Fixed object": "Fixed Object",
    "Trolley car": "Trolley",
    "M/C and PED": "Motorcycle + Pedestrian",
    "School Bus": "Bus",
    "Front-end loader": "Construction Equipment",
}

df["secondary_vehicle"] = df["secondary_vehicle"].replace(standardization_map)
main_categories = {
    "Pedestrian": "Pedestrian",
    "Auto": "Auto",
    "Fixed Object": "Fixed Object",
    "Bicycle": "Bicycle",
    "M/C": "Motorcycle",
    "Parked Vehicle": "Parked Vehicle",
    "Bus": "Bus",
    "Dirtbike": "Motorcycle",
    "Scooter": "Motorcycle",
    "Motorcycle": "Motorcycle",
    "Truck": "Truck",
    "Trolley": "Trolley",
    "Ground": "Ground",
    "Parked Trailer": "Trailer",
    "ATV": "Off-road",
    "Moped": "Motorcycle",
    "Construction Equipment": "Commercial Vehicle",
    "Motorcycle + Pedestrian": "Mixed",
}
df["secondary_vehicle"] = df["secondary_vehicle"].map(main_categories)
df["secondary_vehicle"].value_counts()

secondary_vehicle
Pedestrian            319
Auto                  234
Fixed Object          140
Bicycle                28
Motorcycle             26
Parked Vehicle         18
Bus                     5
Truck                   4
Ground                  3
Trailer                 3
Trolley                 2
Mixed                   2
Commercial Vehicle      2
Off-road                1
Name: count, dtype: int64

In [21]:
data_types = {
    'crash_id': 'Int64',
    'year': 'int32',
    'police_district': 'category',
    'crash_date': 'datetime64[ns]',
    'day_name': 'category',
    'primary_street': 'string',
    'age': 'Float64',
    'victim_gender': 'category',
    'collision_type': 'string',
    'hit_and_run': 'bool',
    'primary_vehicle': 'category',
    'secondary_vehicle': 'category',
    'arrest_made': 'category',
    'investigation_status': 'string',
    'time': 'object',
    'latitude': 'float64',
    'longitude': 'float64',
    'age_group': 'category'
}
df = df.astype(data_types)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   crash_id              797 non-null    Int64         
 1   year                  797 non-null    int32         
 2   police_district       797 non-null    category      
 3   crash_date            797 non-null    datetime64[ns]
 4   day_name              797 non-null    category      
 5   primary_street        797 non-null    string        
 6   age                   792 non-null    Float64       
 7   victim_gender         796 non-null    category      
 8   collision_type        797 non-null    string        
 9   hit_and_run           797 non-null    bool          
 10  primary_vehicle       797 non-null    category      
 11  secondary_vehicle     787 non-null    category      
 12  arrest_made           781 non-null    category      
 13  investigation_status

In [22]:
df.to_csv("/kaggle/working/philadelphia_fatal_crashes_clean.csv")

In [23]:
df.sample(5)

,crash_id,year,police_district,crash_date,day_name,primary_street,age,victim_gender,collision_type,hit_and_run,primary_vehicle,secondary_vehicle,arrest_made,investigation_status,time,latitude,longitude,age_group
722,50226,2024,19,2024-09-26,Thursday,66th St.,14.0,F,Unit #1 was traveling east on Haverford Ave. a...,True,Auto,Pedestrian,No,Active investigation,21:38:00,39.972348,-75.248872,0-18
270,49775,2021,24,2021-03-19,Friday,Erie Ave.,39.0,F,Pedestrian was crossing Erie Ave. when struck ...,True,Auto,Pedestrian,Pending,Reviewing charges,NaN,40.005401,-75.106656,36-45
320,49824,2021,39,2021-08-08,Sunday,Kelly Drive,31.0,M,"M/C S/B on Kelly Drive, lost control. Struck g...",False,Motorcycle,Auto,No,No arrest. Driver deceased.,NaN,40.008293,-75.194539,26-35
652,50157,2024,26,2024-03-13,Wednesday,Lehigh Ave.,48.0,F,Unit #1 was traveling east on Lehigh Ave. cros...,False,Motorcycle,Auto,No,Active investigation,19:22:00,39.987720,-75.123587,46-55
235,49741,2020,25,2020-12-12,Saturday,Erie Ave,40.0,M,"Unit #1, illegal ATV, travelling E/B on Erie, ...",False,Off-road,Auto,Pending,Under Investigation,NaN,40.008762,-75.147750,36-45
